### Fetch Election Results

In [1]:
# import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import geopandas as gpd

In [2]:
## set view settings
pd.options.display.max_columns = 400
pd.options.display.max_rows = 400

#### Step 1: Download the webpage and store it 

In [3]:
# store the 2022 general elections results page
url = 'https://enr.electionsfl.org/LEO/3264/Precincts/44740/'

# get the webpage
response = requests.get(url)

# create a beautifulsoup object using the webpage html
soup = BeautifulSoup(response.text)

#### Step 2: Identify elements that will be extracted. Use the 'inspect' tool to do this or read the HTML stored in the 'soup' variable

In [4]:
# precinct name - .PrecinctName
# choices - .ChoiceColumn
# vote percentages - .col-xs-4+ .notranslate
# votes per choice - .col-xs-2+ .ResultsColumn
# total votes - .TotalColumn

#### Step 3: Extract data

In [14]:
# create empty dataframe
df = pd.DataFrame(columns={'precinct', 'choices', 'pct', 'votes', 'totalVotes'})

In [15]:
# create list of precincts

precinctsList = [] 

names = soup.select(".PrecinctName") 
for name in names:
    precinctsList.append(name.text)

In [16]:
# create list of choices
choicesList = ['Ron DeSantis', 'Charlie Crist', 'Hector Roos', 'Carmen Jackie Gimenez']*136

In [17]:
# create list of vote percentages per choice

percentagesList = []

percentages = soup.select(".col-xs-4+ .notranslate") 
for percentage in percentages:
    percentagesList.append(round(float(percentage.text.split('%')[0])/100, 4))

In [18]:
# create list of votes per choice

votesList = []

votes = soup.select(".col-xs-2+ .ResultsColumn") 
for vote in votes:
    votesList.append(vote.text)

In [19]:
# create list of total votes

totalVotesList = []
totalVotes = soup.select(".TotalColumn") 
for totalVote in totalVotes:
    totalVotesList.append(totalVote.text)
    
totalVotesList = np.repeat(totalVotesList,4)

#### Step 4: Assemble dataframe

In [20]:
# add precincts
df['precinct'] = precinctsList

# replicate rows
df = df.iloc[np.arange(len(df)).repeat(4)].reset_index(drop=True)

In [21]:
# add votes
df['votes'] = votesList

In [22]:
# add vote percentages
df['pct'] = percentagesList

In [37]:
# add votes per choice
df['choices'] = choicesList

In [23]:
# add total votes
df['totalVotes'] = totalVotesList

#### Step 5: Clean & export dataframe

In [24]:
# turn totalVotes column into int
df['totalVotes'] = df["totalVotes"].replace(",", "", regex=True).astype(int)

In [25]:
# format str columns to ints
df['votes'] = df['votes'].replace(",", "", regex=True).astype(int)
df['precinct'] = df['precinct'].astype(int)

In [40]:
# filter candidate & save as new dataframe - CRIST
crist = df.query(" choices == 'Charlie Crist' ").sort_values(by="pct", ascending=False).reset_index(drop=True)

In [39]:
# filter candidate & save as new dataframe - DESANTIS
desantis = df.query(" choices == 'Ron DeSantis' ").sort_values(by="pct", ascending=False).reset_index(drop=True)

#### Step 6: Merge Votes Data w/ Precincts Shapes

In [41]:
# geojson file of City precincts in Leon County
# source: https://geodata-tlcgis.opendata.arcgis.com/datasets/election-precincts-2022/explore?location=30.479225%2C-84.346550%2C9.28
precincts = gpd.read_file("../data/2022-Precincts.geojson")

In [42]:
# change 'precinict' column type to integer
precincts['PRECINCT'] = precincts['PRECINCT'].astype(int)

In [43]:
precincts[['POLLLOC', 'PRECINCT', 'geometry']].merge(desantis, left_on="PRECINCT", right_on="precinct")[['POLLLOC', 'precinct', 'votes', 'choices', 'pct', 'totalVotes', 'geometry']].sort_values(by='precinct').to_file("../data/2022-general-desantis-results.geojson", driver='GeoJSON')

In [44]:
precincts[['POLLLOC', 'PRECINCT', 'geometry']].merge(crist, left_on="PRECINCT", right_on="precinct")[['POLLLOC', 'precinct', 'votes', 'choices', 'pct', 'totalVotes', 'geometry']].sort_values(by='precinct').to_file("../data/2022-general-crist-results.geojson", driver='GeoJSON')